训练前，务必选择一个fold（范围：1-4）

In [1]:
# fold = 4  # 1表示在234上训练，1上验证；以此类推
# if fold == 1:
#     tr_set = 'tr234'
# elif fold == 2:
#     tr_set = 'tr134'
# elif fold == 3:
#     tr_set = 'tr124'
# elif fold == 4:
#     tr_set = 'tr123'

# 设定其余参数
tr_set = 'try'
fold=0
_boosting_type='gbdt'
_num_leaves=32
_max_depth=-1
_n_estimators=5000
_subsample=0.8
_colsample_bytree=0.8
_learning_rate=0.05
_min_child_weight=1

test_set='A'  # 1为A赛段，2为B赛段

运行以下单元格，并获取训练的predict预测文件和模型文件

In [2]:
import lightgbm as lgb
import pandas as pd
import numpy as np
from scipy import sparse
import feather
from sklearn.feature_selection import SelectFromModel
import time
import matplotlib.pyplot as plt

def get_data(stage):
#     if stage == 1:
#         tr_y_path = 'data/output/tr1/tr234_y.csv'
#         val_y_path = 'data/output/tr1/tr1_val_y.csv'
#         tr_x_path = 'data/output/tr1/tr234_9_2162.npz'
#         val_x_path = 'data/output/tr1/tr1_val_9_2162.npz'
#     elif stage == 2:
#         tr_y_path = 'data/output/tr2/tr134_y.csv'
#         val_y_path = 'data/output/tr2/tr2_val_y.csv'
#         tr_x_path = 'data/output/tr2/tr134_9_2162.npz'
#         val_x_path = 'data/output/tr2/tr2_val_9_2162.npz'
#     elif stage == 3:
#         tr_y_path = 'data/output/tr3/tr124_y.csv'
#         val_y_path = 'data/output/tr3/tr3_val_y.csv'
#         tr_x_path = 'data/output/tr3/tr124_9_2162.npz'
#         val_x_path = 'data/output/tr3/tr3_val_9_2162.npz'
#     elif stage == 4:
#         tr_y_path = 'data/output/tr4/tr123_y.csv'
#         val_y_path = 'data/output/tr4/tr4_val_y.csv'
#         tr_x_path = 'data/output/tr4/tr123_9_2162.npz'
#         val_x_path = 'data/output/tr4/tr4_val_9_2162.npz'
#     else:
#         return None

    tr_y_path = 'data/tr_y.f'
    val_y_path = 'data/val_y.f'
    tr_x_path = 'data/tr_x.f'
    val_x_path = 'data/val_x.f'
    print('reading data from:')
    tr_y = feather.read_dataframe(tr_y_path)
    print(tr_y_path)
    val_y = feather.read_dataframe(val_y_path)
    print(val_y_path)

    tr_x = feather.read_dataframe(tr_x_path)
    print(tr_x_path)
    val_x = feather.read_dataframe(val_x_path)
    print(val_x_path)
    
    print('tr_x:', tr_x.shape)
    print('val_x:', val_x.shape)
    print('tr_y:', tr_y.shape)
    print('val_y:', val_y.shape)
    
    return tr_y, val_y, tr_x, val_x


In [12]:
# 读取数据
tr_y, val_y, tr_x, val_x = get_data(0)
t1 = time.time()
# 训练
print('start time:', time.asctime(time.localtime(t1)))
clf = lgb.LGBMClassifier(
    boosting_type=_boosting_type, num_leaves=_num_leaves, reg_alpha=0.0, reg_lambda=1,
    max_depth=_max_depth, n_estimators=_n_estimators, objective='binary',
    subsample=_subsample, colsample_bytree=_colsample_bytree, subsample_freq=1,
    learning_rate=_learning_rate, min_child_weight=_min_child_weight, device='cpu', random_state=2018
)
print('fit...')
clf.fit(tr_x, tr_y, eval_set=[(val_x, val_y)], eval_metric='auc',early_stopping_rounds=50)
print('training completed!')
t2 = time.time()
print('end time:', time.asctime(time.localtime(t2)))
cost = t2-t1
tr_hour = int(cost//(60*60))
tr_miniute = int(cost%(60*60)/60)
print('time cost:{}h  {}m'.format(tr_hour, tr_miniute))

# 生成结果
last_score = float('%.6f' % clf.best_score_['valid_0']['auc'])
last_score = int(last_score*1000000)

print('saving model to data/model/{}_ml{}_mcw{}_sa{}_cs{}_{}_{}____if1_f2162_{}_{}'.format(tr_set,_num_leaves,_min_child_weight,_subsample,_colsample_bytree,_n_estimators,last_score,tr_hour, tr_miniute))
clf.booster_.save_model('data/model/{}_ml{}_mcw{}_sa{}_cs{}_{}_{}____if1_f2162_{}_{}'.format(tr_set,_num_leaves,_min_child_weight,_subsample,_colsample_bytree,_n_estimators,last_score,tr_hour, tr_miniute))

reading data from:
data/tr_y.f
data/val_y.f
data/tr_x.f
data/val_x.f
tr_x: (792004, 298)
val_x: (198002, 298)
tr_y: (792004, 1)
val_y: (198002, 1)
start time: Thu Jun 14 00:27:56 2018
fit...


C:\Users\bhneo\Anaconda3\envs\lgb\lib\site-packages\sklearn\preprocessing\label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\bhneo\Anaconda3\envs\lgb\lib\site-packages\sklearn\preprocessing\label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


KeyboardInterrupt: 

In [4]:
booster = lgb.Booster(model_file='data/model/try_ml32_mcw1_sa0.8_cs0.8_5000_992704____if1_f2162_0_4')

In [13]:
print('reading test{}...'.format(test_set))
test = feather.read_dataframe('data/test{}.f'.format(test_set))
test_x = test.drop(columns=['id'])

print('test shape:', test_x.shape)
print('predicting...')
t1 = time.time()
print('start time:', time.asctime(time.localtime(t1)))
# test_x['score'] = clf.predict_proba(test_x)[:, 1]
test['score'] = booster.predict(test_x)
# del test_x
# test['score'] = test['score'].apply(lambda x: float('%.6f' % x))
# t2 = time.time()
# print('end time:', time.asctime(time.localtime(t2)))
# cost = t2-t1
# hour = int(cost//(60*60))
# miniute = int(cost%(60*60)/60)
# print('time cost:{}h  {}m'.format(hour, miniute))
# print('saving submission to data/result/test{}_predict{}_{}.csv'.format(test_set, fold, last_score))
# test[['id','score']].to_csv('data/result/test{}_predict{}_{}.csv'.format(test_set, fold, last_score), index=False)
# del test
# print('completed!')



reading testA...
test shape: (491668, 298)
predicting...
start time: Thu Jun 14 15:51:11 2018


In [14]:
test[['id','score']]

,id,score
0,8e8290c270ec4bc3448dd5edd35c6f059b42d38f9ddd6f...,0.000197
1,f36c5fa5c0e7afccf733a4d74c7e06ffe43cc8fd24eda7...,0.001469
2,ee85f808b1fd49eaba308527e1686c509dc8e3e5057488...,0.000009
3,0df905aa187938d56a9b0816b13f54ac1f87d658a33cd0...,0.000749
4,15f532f979c4f092bbbe28e5409c8c3b8454ece8f1ab3e...,0.000058
5,614728f2aba299d20ddffca7e1ebc38ae8f239663562a9...,0.003171
6,3dc22d969895bc4afdaa5266d1586ae1a360adfcc638b5...,0.000174
7,b6cba0173d6a894f443578b5d82c00444dfe24c5e1705d...,0.000002
8,84d010a48c747947588589473e2ebdc59db3d348849c5b...,0.000017
9,bc00fec7ff8b3e4f8bd826048da109e7fe40fcc604ba28...,0.002202


In [16]:
test[['id','score']].drop_duplicates('id')


,id,score
0,8e8290c270ec4bc3448dd5edd35c6f059b42d38f9ddd6f...,0.000197
1,f36c5fa5c0e7afccf733a4d74c7e06ffe43cc8fd24eda7...,0.001469
2,ee85f808b1fd49eaba308527e1686c509dc8e3e5057488...,0.000009
3,0df905aa187938d56a9b0816b13f54ac1f87d658a33cd0...,0.000749
4,15f532f979c4f092bbbe28e5409c8c3b8454ece8f1ab3e...,0.000058
5,614728f2aba299d20ddffca7e1ebc38ae8f239663562a9...,0.003171
6,3dc22d969895bc4afdaa5266d1586ae1a360adfcc638b5...,0.000174
7,b6cba0173d6a894f443578b5d82c00444dfe24c5e1705d...,0.000002
8,84d010a48c747947588589473e2ebdc59db3d348849c5b...,0.000017
9,bc00fec7ff8b3e4f8bd826048da109e7fe40fcc604ba28...,0.002202


In [11]:
last_score=0
# del test_x
# test['score'] = test['score'].apply(lambda x: float('%.6f' % x))
t2 = time.time()
print('end time:', time.asctime(time.localtime(t2)))
cost = t2-t1
hour = int(cost//(60*60))
miniute = int(cost%(60*60)/60)
print('time cost:{}h  {}m'.format(hour, miniute))
print('saving submission to data/result/test{}_predict{}_{}.csv'.format(test_set, fold, last_score))
test[['id','score']].to_csv('data/result/test{}_predict{}_{}.csv'.format(test_set, fold, last_score), index=False)
del test
print('completed!')

end time: Thu Jun 14 15:46:27 2018
time cost:0h  2m
saving submission to data/result/testA_predict0_0.csv
completed!
